<span style="color: green;">

# Introduction

In this laboratory we will get our hands dirty working with Large Language Models (e.g. GPT and BERT) to do various useful things. I you haven't already, it is highly recommended to:

- Read the [Attention is All you Need](https://arxiv.org/abs/1706.03762) paper, which is the basis for all transformer-based LLMs.
- Watch (and potentially _code along_) with this [Andrej Karpathy video](https://www.youtube.com/watch?v=kCc8FmEb1nY) which shows you how to build an autoregressive GPT model from the ground up.

# Exercise 1: Warming Up

In this first exercise you will train a _small_ autoregressive GPT model for character generation (the one used by Karpathy in his video) to generate text in the style of Dante Aligheri. Use [this file](https://archive.org/stream/ladivinacommedia00997gut/1ddcd09.txt), which contains the entire text of Dante's Inferno (**note**: you will have to delete some introductory text at the top of the file before training). Train the model for a few epochs, monitor the loss, and generate some text at the end of training. Qualitatively evaluate the results
</span>


## Pipeline

- scarica il file
- elimina la parte iniziale per fare il training
- tokenizza il testo
- crea il modello
- training (per poche epoche)
- guarda la loss
- genera testo e valuta qualitativamente i risultati


In [ ]:
# conda create --name Lab2_DLA
# conda install pytorch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 pytorch-cuda=11.8 -c pytorch -c nvidia
# conda install -c conda-forge transformers
# pip install scikit-learn
# pip install evaluate
# pip install transformers[torch]

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F


import os
import matplotlib.pyplot as plt
import numpy as np

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GenerationConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import GPT2ForSequenceClassification
from transformers import TrainingArguments, Trainer

from datasets import load_dataset
import evaluate

/home/dl23vitcas/anaconda3/envs/Lab2_DLA/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
i = 0
if torch.cuda.is_available():
    device = torch.device("cuda:" + str(i))  # cuda:0 , cuda:1
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # yes, i have a MacBookPro with M1 Pro
else:
    device = torch.device("cpu")

print(f"device: {device}")
print(
    "GPU: " + torch.cuda.get_device_name(i)
    if device == torch.device("cuda:" + str(i))
    else ""
)

device: cuda:0
GPU: NVIDIA RTX A2000 12GB


In [ ]:
batch_size = 64  # how many indipendent sequences will be process in parallel ?
# what is the maximum context length for predictions ?
block_size = 256  # contect_lenght
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [ ]:
torch.manual_seed(1337)

In [ ]:
print("dataset\n")
with open("input.txt", "r", encoding="utf-8") as f:
    text = f.read()
print(f"lenght of dataset in characters: {len(text)}\n")

chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'chars: {"".join(chars)}')
print(f"vocab_size: {vocab_size}")
# print(text[:400])

In [ ]:
# create a mapping from character to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# encoder: take a string, output a list of integer
encode = lambda s: [stoi[c] for c in s]
# decoder: take a list of integer, output a string
decode = lambda l: "".join(itos[i] for i in l)

print(encode("Ciao Vittorio"))
print(decode(encode("Ciao Vittorio")))

In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:200])
# the 200 characters we looked at earlier will to the GPT look like this

In [ ]:
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
print(train_data.shape)
print(train_data[:10])
print(f"Context lenght: {block_size}")

train_data[: block_size + 1]  # chunk of data

In [ ]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]
print(f"Context lenght: {block_size}")
for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

    if t == 10:
        break

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(low=0, high=len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x = x.to(device)
    y = y.to(device)
    return x, y


xb, yb = get_batch("train")

print("input: ")
print(f"{xb.shape}\n{xb}")
print("target: ")
print(f"{yb.shape}\n{yb}")

print("---")

for b in range(batch_size):  # 0 --> 4
    for t in range(block_size):  # 0 --> 8
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")
    break

In [ ]:
# Model

torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(
            num_embeddings=vocab_size, embedding_dim=n_embd
        )
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        self.blocks = nn.Sequential(
            *[Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        )
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(
            torch.arange(T, device=device)
        )  # (T, C )
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)  # (B, T, C)
        x = self.ln_f(x)
        logits = self.lm_head(x)  # (B,T, vocab_size)
        # n.b: cross entropy vuole i logits con shape (B * T, C) e non (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    # function to generate from the model
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context

        for _ in range(max_new_tokens):
            # crop idx to the last block_size torkens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)  # logits.shape = (B * T, C)
            # focus only in the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append samples index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


class Head(nn.Module):
    """one head of sel-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)  # (B, T, C)
        q = self.query(x)  # (B, T, C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1)  # (B, T, C) @ (B, C, T) --> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B,T,C) --> (B, T, C)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallell"""

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Trasformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
model = BigramLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters()) / 1e6, "M parameters")

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
os.makedirs("./text_generation_model/", exist_ok=True)

path_models = "./text_generation_model/model.pth"

if not os.path.exists(path_models):  # se non esiste fai l'addestramento
    for iter in range(max_iters):
        # every once in a while evaluate the loss on train and val sets
        if iter % eval_interval == 0:
            losses = estimate_loss(model)
            print(
                f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
            )
        xb, yb = get_batch("train")
        logits, loss = model(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()
        torch.save(model.state_dict(), path_models)
else:
    print(f"Model exist on path {path_models}")

In [ ]:
# model.load_state_dict(torch.load(path_models)) # questa istruzione crea problemi se addestro su gpu su papavero e provo a caricare i pesi su mps o cpu, usa torhc.load specificando map_location il dispositivo mps o cpu
# torch.load(path_models, map_location=device)

model.load_state_dict(torch.load(path_models, map_location=device))

max_new_tokens = 5000

# generates text
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=max_new_tokens)[0].tolist()))
# il testo generato a seguito dell'addestramento dovrebbe avere senso

Il testo generato ha una grande somiglianza con lo stile di scrittura del noto poeta Dante. Si può notare come le frasi siano corte e risultano suddivise in gruppi di tre frase (chiamate "terzine"). Ogni terzina ha le due ultime due frasi "indentate" a destra. La maggiorparte delle parole hanno un senso nella lingua italiana, altre invece contengono dei semplici errori di lettere all'interno.
Altra fattore da evidenziare è che il modello spesso riesce a capire che quando siamo all'inizio di una conversazione e viene posto il carattere ":" prima del suo inizio.

Sicuramente con un addestramento con più epoche avremmo un risultato molto migliore ricco di altre caratteristiche dello stile di dante.


<span style="color: green;">

# Exercise 2: Working with Real LLMs

Our toy GPT can only take us so far. In this exercise we will see how to use the [Hugging Face](https://huggingface.co/) model and dataset ecosystem to access a _huge_ variety of pre-trained transformer models.

## Exercise 2.1: Installation and text tokenization

First things first, we need to install the [Hugging Face transformer library](https://huggingface.co/docs/transformers/index):

    conda install -c huggingface -c conda-forge transformers

The key classes that you will work with are `GPT2Tokenizer` to encode text into sub-word tokens, and the `GPT2LMHeadModel`. **Note** the `LMHead` part of the class name -- this is the version of the GPT2 architecture that has the text prediction heads attached to the final hidden layer representations (i.e. what we need to **generate** text).

Instantiate the `GPT2Tokenizer` and experiment with encoding text into integer tokens. Compare the length of input with the encoded sequence length.

**Tip**: Pass the `return_tensors='pt'` argument to the togenizer to get Pytorch tensors as output (instead of lists).
</span>


In [ ]:
# installa HF transformers (DONE)
# è stato necessario eseguire anche --> conda install -c conda-forge huggingface_hub (DONE)
# encode text into sub-word tokens with GPT2Tokenizer (DONE)
# istanzia GPT2Tokenizer e converti il testo in token interi (DONE)
# confronta la lunghezza dell'input (testo) con la lunghezza della sequenza codificata (lunghezza della sequenza di interi) (DONE)
# ricordati di usare return_tensors='pt' in modo da ottenere un tensore torch (DONE)
# usa come modello GPT2LMHeadModel (questo modello ha un'uscita apposita per generare il testo)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# stampa gli attributi e i metodi che posso invocare con l'oggetto tokenizer
# print(*dir(tokenizer), sep="\n")
# print(tokenizer.get_vocab) # important params settings

print(f"Vocab size: {tokenizer.vocab_size}")
print(f"Model max length: {tokenizer.model_max_length}\n")

print(tokenizer.encode("Hi Vittorio"))  # dict with keys: input_ids, attention_mask
print(tokenizer.decode(tokenizer.encode("Hi Vittorio")))

In [ ]:
def view_encoded_text(text, huge=False):
    encoded_text = tokenizer(text, return_tensors="pt")["input_ids"]
    contents = f"{text} --> {encoded_text}" if huge == False else ""
    print(
        f"° length text (in characters): {len(text)} - length encoded text: {encoded_text.shape[1]}\t {contents}"
    )

In [ ]:
print("Esempi:\n")
view_encoded_text("Good Morning!")
view_encoded_text(" Good Morning!")
view_encoded_text("  Good Morning!")
view_encoded_text("   Good Morning!")
view_encoded_text("Good Morning!   ")
view_encoded_text("Good Morning!  ")
view_encoded_text("Good Morning! ")
view_encoded_text("Good Morning!")

Da notare che la lunghezza del testo codificato tiene conto del numero di parole e della punteggiatura (carattere spazio incluso).
Per evitare di scrivere un programma che conti il numero di parole e il numero di caratteri della punteggiatura contiamo i caratteri della testo come avevamo fatto nel precedente esercizio.

Si può notare che all'aumentare/diminuire della lunghezza del testo aumenta/diminuisce la lunghezza della sequenza di interi. Però fra le due lenght non c'è una corrispondenza biunivoca (per esempio ad un testo di 13 caratteri potrebbero corrispondere una codifica con 3 o 4 interi come si può vedere negli esempi sopra).
Si può notare inoltre che l'aggiunta di tanti spazi fa aumentare la lunghezza della sequenza di interi. Questo potrebbe essere uno svantaggio in certi casi. In genere l'aggiunta di più di uno spazio nella scrittura con tastiera qwerty è un'errore di battitura che però un modello di linguaggio codifica come input di testo grezzo e potrebbe creare delle relazioni tra parole successive.

Proviamo con esempi di testo più lunghi.


In [ ]:
view_encoded_text(
    "My name is Vittorio Casula. I'm 26 years old. I graduated in Computer Engineering in 2021 at University of Florence.",
    huge=True,
)
view_encoded_text(
    "My best purchase of the last 2 years is my Macbook Pro with M1 Pro Processor. Thanks to it, I'm able to run fast my Machine Learning Script on my laptop.",
    huge=True,
)
view_encoded_text(
    "However, I believe that the optimal setup for working in the Machine Learning field should involve two devices: the first being a laptop (not necessarily a MacBook) with a large-capacity battery, and the second being a server (connected via SSH) on which you can run your scripts without utilizing the resources of the laptop.",
    huge=True,
)

In [ ]:
# proviamo a fare una stringa dinamica che diventa sempre più lunga

dynamic_str = ""
len_max = 1024
chars = f"ABCDEFGHILMNOPQRSTUVXZabcdefghijlmnopqrstuvxz0123456789"
indices = torch.randint(low=0, high=len(chars), size=(len_max,)).tolist()

lengths_text = []
lengths_encoded_text = []
for i in range(len(indices)):
    dynamic_str += chars[indices[i]]
    lengths_text.append(len(dynamic_str))
    lengths_encoded_text.append(
        tokenizer(dynamic_str, return_tensors="pt")["input_ids"].shape[1]
    )

plt.title("Lunghezza del testo (in caratteri) vs Lunghezza del testo codificato")
plt.xlabel("Lunghezza (in caratteri) del testo")
plt.ylabel("Lunghezza del testo codificato")

plt.plot(lengths_text, lengths_encoded_text)

<span style="color: green;">

## Exercise 2.2: Generating Text

There are a lot of ways we can, given a _prompt_ in input, sample text from a GPT2 model. Instantiate a pre-trained `GPT2LMHeadModel` and use the [`generate()`](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to generate text from a prompt.

**Note**: The default inference mode for GPT2 is _greedy_ which might not results in satisfying generated text. Look at the `do_sample` and `temperature` parameters.
</span>


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

input_text = "My name is Vittorio. I'm AI student at University of Florence."

inputs = tokenizer(input_text, return_tensors="pt")
# print(inputs)
# print(*inputs)  # get the keys of returned dict
# outputs = model.generate(**inputs) # warning: Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
outputs = model.generate(**inputs, pad_token_id=tokenizer.eos_token_id)
# print(outputs)
print(f"Generated text: \n{tokenizer.decode(outputs[0])}")
# if you want decode multiple interger sequence you should use batch_decode()

In [ ]:
# interesting warning:Using the model-agnostic default `max_length` (=20) to control thegeneration length.
# We recommend setting `max_new_tokens` to control the maximum length of the generation.

outputs = model.generate(
    **inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100
)
# print(outputs)
print("Generated text: \n")
print(tokenizer.decode(outputs[0]))

Come vediamo il testo generato contiene il testo iniziamente codificato e una frase generata che si ripete fino a raggiungere il numero massimo di token specificato. Non ha molto senso questo testo generato.


In [ ]:
# check "do_sample" and "temperature" parameters
# link: https://huggingface.co/docs/transformers/generation_strategies

In [ ]:
print(model.generation_config)
# Printing out the model.generation_config reveals only the values that are different from the default generation configuration,
# and does not list any of the default values.

# The default generation configuration limits the size of the output combined with the input prompt to a maximum of 20 tokens
# to avoid running into resource limitations. The default decoding strategy is greedy search, which is the simplest decoding
# strategy that picks a token with the highest probability as the next token. For many tasks and small output sizes this works well.
# However, when used to generate longer outputs, greedy search can start producing highly repetitive results.

### Customize Text Generation


do_sample: if set to True, this parameter enables decoding strategies such as multinomial sampling, beam-search multinomial sampling,
Top-K sampling and Top-p sampling. All these strategies select the next token from the probability distribution over the entire vocabulary
with various strategy-specific adjustments.


In [ ]:
outputs = model.generate(
    **inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=100, do_sample=True
)
# print(outputs)
print("Generated text: \n")
print(tokenizer.decode(outputs[0]))

In [ ]:
generation_config = GenerationConfig(
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

outputs = model.generate(**inputs, generation_config=generation_config)
# print(outputs)
print("Generated text: \n")
print(tokenizer.decode(outputs[0]))

Assisted decoding is a modification of the decoding strategies above that uses an assistant model with the same tokenizer (ideally a much smaller model) to greedily generate a few candidate tokens. The main model then validates the candidate tokens in a single forward pass, which speeds up the decoding process. Currently, only greedy search and sampling are supported with assisted decoding, and doesn’t support batched inputs. To learn more about assisted decoding, check this blog post.


When using assisted decoding with sampling methods, you can use the temperature argument to control the randomness just like in multinomial sampling. However, in assisted decoding, reducing the temperature will help improving latency.


In [ ]:
temperature = [0.2, 0.4, 0.6, 0.8, 1]
for temp in temperature:
    outputs = model.generate(
        **inputs, generation_config=generation_config, do_sample=True, temperature=temp
    )
    # print(outputs)
    print(f"\n° Generated text (temperature = {temp}):")
    print(f"{tokenizer.decode(outputs[0])}")

Testo generato molto vario al crescere del parametro temperature. Per piccoli valori di temperature si ha il fenomeno della ripetizione del testo.


<span style="color: green;">

# Exercise 3: Reusing Pre-trained LLMs (choose one)

Choose **one** of the following exercises (well, _at least_ one). In each of these you are asked to adapt a pre-trained LLM (`GPT2Model` or `DistillBERT` are two good choices) to a new Natural Language Understanding task. A few comments:

- Since GPT2 is a _autoregressive_ model, there is no latent space aggregation at the last transformer layer (you get the same number of tokens out that you give in input). To use a pre-trained model for a classification or retrieval task, you should aggregate these tokens somehow (or opportunistically select _one_ to use).

- BERT models (including DistillBERT) have a special [CLS] token prepended to each latent representation in output from a self-attention block. You can directly use this as a representation for classification (or retrieval).

- The first _two_ exercises below can probably be done _without_ any fine-tuning -- that is, just training a shallow MLP to classify or represent with the appropriate loss function.

# Exercise 3.1: Training a Text Classifier (easy) <span style="color: red;">(DONE)</span>

Peruse the [text classification datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=downloads). Choose a _moderately_ sized dataset and use a LLM to train a classifier to solve the problem.

**Note**: A good first baseline for this problem is certainly to use an LLM _exclusively_ as a feature extractor and then train a shallow model.

# Exercise 3.2: Training a Question Answering Model (harder)

Peruse the [multiple choice question answering datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:multiple-choice&sort=downloads). Chose a _moderately_ sized one and train a model to answer contextualized multiple-choice questions. You _might_ be able to avoid fine-tuning by training a simple model to _rank_ the multiple choices (see margin ranking loss in Pytorch).

# Exercise 3.3: Training a Retrieval Model (hardest)

The Hugging Face dataset repository contains a large number of ["text retrieval" problems](https://huggingface.co/datasets?task_categories=task_categories:text-retrieval&p=1&sort=downloads). These tasks generally require that the model measure _similarity_ between text in some metric space -- naively, just a cosine similarity between [CLS] tokens can get you pretty far. Find an interesting retrieval problem and train a model (starting from a pre-trained LLM of course) to solve it.

**Tip**: Sometimes identifying the _retrieval_ problems in these datasets can be half the challenge. [This dataset](https://huggingface.co/datasets/BeIR/scifact) might be a good starting point.
</span>


model selected: GPT2


In [2]:
name_model_pretrained = "gpt2"

Dataset scelto: tweet_eval (link: https://huggingface.co/datasets/tweet_eval)

TweetEval consists of seven heterogenous tasks in Twitter, all framed as multi-class tweet classification. The tasks include - irony, hate, offensive, stance, emoji, emotion, and sentiment. All tasks have been unified into the same benchmark, with each dataset presented in the same format and with fixed training, validation and test splits.
The text in the dataset is in English, as spoken by Twitter users.

selected task: emoji


In [3]:
dataset = load_dataset("tweet_eval", "emoji")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})


In [4]:
# some example
dataset["train"][:5]

{'text': ['Sunday afternoon walking through Venice in the sun with @user ️ ️ ️ @ Abbot Kinney, Venice',
  "Time for some BBQ and whiskey libations. Chomp, belch, chomp! (@ Lucille's Smokehouse Bar-B-Que)",
  'Love love love all these people ️ ️ ️ #friends #bff #celebrate #blessed #sundayfunday @ San…',
  '️ ️ ️ ️ @ Toys"R"Us',
  'Man these are the funniest kids ever!! That face! #HappyBirthdayBubb @ FLIPnOUT Xtreme'],
 'label': [12, 19, 0, 0, 2]}

### Data Fields

For emoji config:

- text: a string feature containing the tweet
- label: an int classification label with the following mapping


![labels](./img/labels.png)


In [5]:
id2label = {
    0: "Red Heart",
    1: "Smiling Face with Heart-Eyes",
    2: "Face with Tears of Joy",
    3: "Two Hearts",
    4: "Fire",
    5: "Smiling Face with Similing Eyes",
    6: "Similing Face with Sunglasses",
    7: "Sparkles",
    8: "Blue Heart",
    9: "Face Blowing a Kiss",
    10: "Camera",
    11: "Flag United States",
    12: "Sun",
    13: "Purple Heart",
    14: "Winking Face",
    15: "Hundred Points",
    16: "Beaming Face with Smiling Eyes",
    17: "Christmas Tree",
    18: "Camera with Flash",
    19: "Winking Face with Tongue",
}

label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(name_model_pretrained)
tokenizer.pad_token = tokenizer.eos_token
print(tokenizer)

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)


Si può notare che a partire dal modello preaddestrato con la classe AutoTokenizer si può ricavare il Tokenizer usato per l'addestramento di tale modello.


In [7]:
# funzione per calcolare l'accuracy (viene usata la funzione di sk-learn)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


# funzione per effettuare la tokenizzazione di tutto il dataset (di tutti gli split)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)


tokenized_tweet_eval = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
accuracy = evaluate.load("accuracy")

Con le istruzioni seguenti settiamo il mostro modello pre-addestrato GPT2
Viene impostato l'ID del token di padding (pad_token_id) del modello uguale all'ID del token di fine frase (eos_token_id) del modello.

Il token di padding viene utilizzato nelle operazioni di elaborazione del linguaggio naturale, in particolare durante il padding delle sequenze di testo di lunghezza variabile in modo che abbiano tutte la stessa lunghezza.


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    name_model_pretrained, num_labels=num_labels, id2label=id2label, label2id=label2id
)
model.config.pad_token_id = model.config.eos_token_id

model.name_or_path

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'gpt2'

In [9]:
training_args = TrainingArguments(
    output_dir="./text_classification_model/",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    optim="adamw_torch"
    # no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_tweet_eval["train"],
    eval_dataset=tokenized_tweet_eval["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/dl23vitcas/anaconda3/envs/Lab2_DLA/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,2.118800,2.575381,0.244800
2,1.871900,2.448047,0.259200
3,1.785200,2.435857,0.271200


/home/dl23vitcas/anaconda3/envs/Lab2_DLA/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/dl23vitcas/anaconda3/envs/Lab2_DLA/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=4221, training_loss=1.9634791276828285, metrics={'train_runtime': 1905.5263, 'train_samples_per_second': 70.847, 'train_steps_per_second': 2.215, 'total_flos': 8070567475322880.0, 'train_loss': 1.9634791276828285, 'epoch': 3.0})

In [24]:
model.device

device(type='cuda', index=0)

In [23]:
trainer.model.name_or_path

'gpt2'

In [26]:
trainer.evaluate(eval_dataset=tokenized_tweet_eval["test"])  # on test set

/home/dl23vitcas/anaconda3/envs/Lab2_DLA/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 1.8104512691497803,
 'eval_accuracy': 0.44628,
 'eval_runtime': 165.2223,
 'eval_samples_per_second': 302.623,
 'eval_steps_per_second': 9.46,
 'epoch': 3.0}

## Inference


In [32]:
def inference(sequence_to_classify):
    # path_models = "./text_classification_model/checkpoint-4221/"
    # model = AutoModelForSequenceClassification.from_pretrained(path_models)
    model = trainer.model
    tokenizer = AutoTokenizer.from_pretrained(name_model_pretrained)
    encoded_sequence = tokenizer(sequence_to_classify, return_tensors="pt")
    encoded_sequence = encoded_sequence.to(device)
    logits = model(**encoded_sequence).logits
    predicted_class = logits.argmax().item()
    print(
        f"Classe predetta: {predicted_class} ({id2label[predicted_class]}) - score = {torch.softmax(logits, dim=1).squeeze(0)[predicted_class]}"
    )

In [38]:
inference("i love you")
inference("i missed you")
inference("Happy new Year!")
inference("Merry Christmas!")
inference("Do you take a photo together?")
inference("I'd rather burn myself")
inference("fuck you")
inference("make america great again")
inference("max points")
inference("highest quality")
inference("best quality")

Classe predetta: 0 (Red Heart) - score = 0.24434871971607208
Classe predetta: 0 (Red Heart) - score = 0.2493310272693634
Classe predetta: 5 (Smiling Face with Similing Eyes) - score = 0.3650968074798584
Classe predetta: 17 (Christmas Tree) - score = 0.48969411849975586
Classe predetta: 2 (Face with Tears of Joy) - score = 0.22530613839626312
Classe predetta: 7 (Sparkles) - score = 0.202946737408638
Classe predetta: 2 (Face with Tears of Joy) - score = 0.5167744159698486
Classe predetta: 11 (Flag United States) - score = 0.6926225423812866
Classe predetta: 2 (Face with Tears of Joy) - score = 0.28364524245262146
Classe predetta: 7 (Sparkles) - score = 0.17856758832931519
Classe predetta: 0 (Red Heart) - score = 0.3360006511211395
